In [4]:
import pandas as pd
import vaex as ve
import numpy as np
import datetime
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
df_original_reports = pd.read_csv("data-processed/rki-reports.csv")
df_apple = pd.read_csv("data-processed/data_apple_prepared.csv")
df_rki_orig = pd.read_csv('dash/data/data_rki_apple_prepared_dash.csv')
df_jh_world = pd.read_csv('dash/data/data_jhu_world.csv')
# df = pd.read_csv("dash/data/data_rki_apple_prepared_dash.csv")
df_rki_orig['date']=df_rki_orig['date'].astype('datetime64[ns]')
df_rki_orig.index = df_rki_orig.date
df_jh_world['date']=df_jh_world['date'].astype('datetime64[ns]')
df_jh_world.index = df_jh_world.date

In [13]:
%%time
df_jh_world = pd.read_csv('dash/data/data_jhu_world.csv')
df_jh_world['date'] = df_jh_world['date'].astype('datetime64')

CPU times: user 50.6 ms, sys: 11.4 ms, total: 62 ms
Wall time: 66 ms


In [12]:
%%time
ve_jh_world = ve.open('dash/data/data_jhu_world.csv')
ve_jh_world['date'] = ve_jh_world['date'].astype('datetime64')

CPU times: user 75.7 ms, sys: 14.2 ms, total: 89.8 ms
Wall time: 108 ms


In [7]:
%%time
df_jh_world.groupby(by=['land']).agg(['sum'])['confirmed_change']

CPU times: user 234 ms, sys: 10.4 ms, total: 244 ms
Wall time: 361 ms


,sum
land,
Afghanistan,13658.0
Albania,1097.0
Algeria,9133.0
Andorra,763.0
Angola,80.0
...,...
Vietnam,326.0
West Bank and Gaza,442.0
Yemen,282.0


In [8]:
%%time
ve_jh_world['land', 'confirmed_change'].groupby(by='land', agg={'confirmed_change': 'sum'})

CPU times: user 49.3 ms, sys: 13.5 ms, total: 62.8 ms
Wall time: 86.4 ms


#,land,confirmed_change
0,Afghanistan,13658.0
1,Albania,1097.0
2,Algeria,9133.0
3,Andorra,763.0
4,Angola,80.0
...,...,...
181,Vietnam,326.0
182,West Bank and Gaza,442.0
183,Yemen,282.0
184,Zambia,1055.0


In [9]:
df = df_jh_world
aggregate_by_column = 'region_wb'
value_column = 'confirmed_change'

In [10]:
%%time
df = df_jh_world
# Aggregate most recent day's value for regions
current_reg_df_sum = df.loc[df.date == df.date.max(), :].groupby(aggregate_by_column).agg(['sum'])[value_column]
current_reg_df_sum

CPU times: user 104 ms, sys: 4.28 ms, total: 108 ms
Wall time: 149 ms


,sum
region_wb,
East Asia & Pacific,2620.0
Europe & Central Asia,12740.0
European Union,6745.0
Latin America & Caribbean,37865.0
Middle East & North Africa,11366.0
North America,25199.0
South Asia,14328.0
Sub-Saharan Africa,4410.0


In [14]:
%%time
df = ve_jh_world
# df['date'] = df['date'].astype('datetime64')
# Aggregate most recent day's value for regions
current_reg_df_sum = df[df.date == np.datetime64(df.date.max())].groupby(aggregate_by_column, agg={'confirmed_change': 'sum'}) #.sort(by='confirmed_change', ascending=False)
current_reg_df_sum


CPU times: user 34.8 ms, sys: 4.07 ms, total: 38.9 ms
Wall time: 42.6 ms


#,region_wb,confirmed_change
0,South Asia,14328
1,Europe & Central Asia,12740
2,Middle East & North Africa,11366
3,Sub-Saharan Africa,4410
4,Latin America & Caribbean,37865
5,East Asia & Pacific,2620
6,European Union,6745
7,North America,25199


In [15]:
%%time
# Aggregate yesterday's value for regions
# df = df_jh_world
# yd_reg_df_sum = df.loc[df.date == df.date.max() - timedelta(days=1), :].groupby(aggregate_by_column).agg(['sum'])[value_column]

df = ve_jh_world
yd_reg_df_sum = df[df.date == np.datetime64(df.date.max()) - np.timedelta64(1,'D') ].groupby(aggregate_by_column, agg={'confirmed_change': 'sum'}) #.sort(by='confirmed_change', ascending=False)
yd_reg_df_sum

CPU times: user 35.7 ms, sys: 3.58 ms, total: 39.2 ms
Wall time: 50.8 ms


#,region_wb,confirmed_change
0,South Asia,12261
1,Europe & Central Asia,12517
2,Middle East & North Africa,10273
3,Sub-Saharan Africa,3170
4,Latin America & Caribbean,43411
5,East Asia & Pacific,1653
6,European Union,10244
7,North America,23564


In [17]:
df = ve_jh_world
value_column = 'confirmed'
aggregate_by_column= 'region_wb'
region = 'European Union'
current_reg_df_sum = df[df.date == np.datetime64(df.date.max())].groupby(aggregate_by_column, agg={value_column: 'sum'}).sort(by=value_column, ascending=False)
# current_reg_df_sum[current_reg_df_sum.region_wb == 'European Union']['confirmed_change']
d = current_reg_df_sum[current_reg_df_sum.region_wb == region][value_column]
d.tolist()[0]

1408717.0